In [ ]:
# Importing csv
df = pd.read_csv("C:\\Users\\Sarthak\\Downloads\\medicine.csv")

##### EDA

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
pivot_table = pd.pivot_table(df, index='Description', columns='Reason', aggfunc='size', fill_value=0)
print(pivot_table)

In [ ]:
pivot_table.head()

In [ ]:
# Count the non-zero values in each row
non_zero_counts = pivot_table.apply(lambda row: (row != 0).sum(), axis=1)

# Create a new DataFrame from the non-zero counts
non_zero_counts_df = pd.DataFrame(non_zero_counts, columns=['Non-Zero Count']).reset_index()

# rename the columns 
non_zero_counts_df.columns = ['Description', 'Non-Zero Count']

In [ ]:
non_zero_counts_df[non_zero_counts_df["Non-Zero Count"] != 1]

In [ ]:
# List of values to drop
values_to_drop = non_zero_counts_df[non_zero_counts_df["Non-Zero Count"] != 1]["Description"].tolist()

# Drop rows where 'Description' column contains any of the listed values
df_cleaned = df[~df['Description'].isin(values_to_drop)]

# Optionally, reset the index if needed
df_cleaned = df_cleaned.reset_index(drop=True)

# Display the cleaned DataFrame
print(df_cleaned)

In [ ]:
df_cleaned.info()

In [ ]:
len(df_cleaned["Drug_Name"].unique())

In [ ]:
len(df_cleaned["Reason"].unique())

In [ ]:
len(df_cleaned["Description"].unique())

##### Cosine similarity

In [ ]:
# Encode 'Gender' and 'Medical Condition'
label_encoder = LabelEncoder()

# encoding columns 
df_cleaned['Reason_e'] = label_encoder.fit_transform(df_cleaned['Reason'])
df_cleaned['Description_e'] = label_encoder.fit_transform(df_cleaned['Description'])

# Creating feature matrix using Reason and description column 
features = df_cleaned[['Reason_e','Description_e']]

# Cosine similarity
similarity_matrix = cosine_similarity(features)
similarity_matrix

In [ ]:
df_cleaned.head()

In [ ]:
df_cleaned.info()

In [ ]:
def recommend_drugs(index, similarity_matrix, df_cleaned, n_recommendations=3):
    # Find the similarity scores for the specified index (drug)
    similarity_scores = list(enumerate(similarity_matrix[index]))
    
    # Sort drugs by similarity score in descending order
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices of the top 'n_recommendations' similar drugs (excluding the current drug itself)
    similar_drugs = [drug for drug in similarity_scores if drug[0] != index][:n_recommendations]
    
    # Get drug names from similar drugs
    recommended_drugs = []
    for drug in similar_drugs:
        similar_drug_index = drug[0]
        drug_name = df_cleaned.iloc[similar_drug_index]['Drug_Name']
        recommended_drugs.append(drug_name)
    
    return recommended_drugs

# Example: Recommend drugs for the entry with index = 10
index = 3
recommendations = recommend_drugs(index, similarity_matrix, df_cleaned)
print(f"Recommended Drugs for entry with index {index}: {recommendations}")

In [ ]:
def evaluate_recommendations(index, recommendations, df_cleaned):
    # Get the Reason and Description for the original drug
    original_reason = df_cleaned.iloc[index]['Reason']
    original_description = df_cleaned.iloc[index]['Description']
    original_drug = df_cleaned.iloc[index]['Drug_Name']
    
    print(f"Original Drug for index {index}: {original_drug}")
    print(f"Reason: {original_reason}")
    print(f"Description: {original_description}\n")
    
    # Display the recommended drugs with their Reason and Description
    print("Recommended Drugs:")
    for drug_name in recommendations:
        # Filter the DataFrame to get the row where 'Drug_Name' matches the recommended drug
        recommended_drug = df_cleaned[df_cleaned['Drug_Name'] == drug_name].iloc[0]  # Get the first match
        
        print(f"\nDrug Name: {recommended_drug['Drug_Name']}")
        print(f"Reason: {recommended_drug['Reason']}")
        print(f"Description: {recommended_drug['Description']}")
evaluate_recommendations(index, recommendations, df_cleaned)